## Basado en
- https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [1]:
import pandas as pd
dataset = pd.read_csv('full_dataset.csv')
dataset.tail()

,date,price_eurusd,open_eurusd,high_eurusd,low_eurusd,change_eurusd,bid,ask,price_sp500,open_sp500,high_sp500,low_sp500,change_sp500
9937,6/27/2022,1.06,1.06,1.06,1.05,0.26,695.55,689.88,3900.11,3920.76,3927.72,3889.66,-0.30
9938,6/28/2022,1.05,1.06,1.06,1.05,-0.61,693.59,685.75,3821.55,3913.00,3945.86,3820.14,-2.01
9939,6/29/2022,1.04,1.05,1.05,1.04,-0.75,692.96,686.23,3818.83,3825.09,3836.50,3799.02,-0.07
9940,6/30/2022,1.05,1.04,1.05,1.04,0.41,692.25,684.76,3785.38,3785.99,3818.99,3738.67,-0.88
9941,7/1/2022,1.04,1.05,1.05,1.04,-0.52,693.69,685.26,3825.33,3781.00,3829.82,3752.10,1.06


In [2]:
import numpy as np

in_seq1 =  dataset["price_sp500"].values
in_seq2 = dataset["price_eurusd"].values
out_seq = dataset["bid"].values

In [3]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [4]:
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
dataset

array([[1.41300e+02, 9.80000e-01, 4.54500e+01],
       [1.42000e+02, 9.90000e-01, 4.54500e+01],
       [1.45300e+02, 9.90000e-01, 4.54500e+01],
       ...,
       [3.81883e+03, 1.04000e+00, 6.92960e+02],
       [3.78538e+03, 1.05000e+00, 6.92250e+02],
       [3.82533e+03, 1.04000e+00, 6.93690e+02]])

In [5]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [6]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

In [7]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [8]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [9]:
# demonstrate prediction
x_input = np.array([[3818.83, 1.04], [3785.38, 1.05], [3825.33, 1.04]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[551.0766]]
